<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# UNICEF Arm 2030 Vision #1: Flood Prediction in Malawi

Southern Malawi experienced major flooding in 2015 and again in 2019 with cyclone Idai. Approximate dates of impact are 13 January 2015 and 14 March 2019, respectively.

We have broken up the map of southern Malawi into approximately 1 km sq rectangles. Each rectangle has a unique ID. Each rectangle has been assigned a "target" value which is the fraction (percentage) of that rectangle that was flooded in 2015.

For this competition, the training data is the flood extent in 2015 in southern Malawi, however, you are encouraged to source other flood data for other nearby regions and other historic floods to train your model. (Just be sure to propose any new datasets that are not listed here to Zindi at zindi@zindi.africa for approval.)

The test data to measure the accuracy of your model is the flood extent in southern Malawi in 2019. 


# My approch

My approch is to use ConvLSTM on small sample of the map for rainfall and elevation

https://medium.com/neuronio/an-introduction-to-convlstm-55c9025563a7

https://github.com/bsautermeister/imseq/blob/master/src/model/frame_prediction.py


In [34]:
# !pip install joblib

In [35]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/floodmalawi/Train.csv
/kaggle/input/floodmalawi/UNICEF Arm 2030 Vision 1 Flood Prediction in Malawi.ipynb
/kaggle/input/floodmalawi/south_malawi_topography.csv
/kaggle/input/floodmalawi/SampleSubmission.csv


In [36]:
import numpy as np
import pandas as pd
import tensorflow as tf 
import matplotlib.pyplot as plt
import joblib

In [37]:
train=pd.read_csv('/kaggle/input/floodmalawi/Train.csv')

train.head()

,X,Y,target_2015,elevation,precip 2014-11-16 - 2014-11-23,precip 2014-11-23 - 2014-11-30,precip 2014-11-30 - 2014-12-07,precip 2014-12-07 - 2014-12-14,precip 2014-12-14 - 2014-12-21,precip 2014-12-21 - 2014-12-28,...,precip 2019-03-24 - 2019-03-31,precip 2019-03-31 - 2019-04-07,precip 2019-04-07 - 2019-04-14,precip 2019-04-14 - 2019-04-21,precip 2019-04-21 - 2019-04-28,precip 2019-04-28 - 2019-05-05,precip 2019-05-05 - 2019-05-12,precip 2019-05-12 - 2019-05-19,LC_Type1_mode,Square_ID
0,34.26,-15.91,0.0,887.764222,0.0,0.0,0.0,14.844025,14.552823,12.237766,...,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0,9,4e3c3896-14ce-11ea-bce5-f49634744a41
1,34.26,-15.90,0.0,743.403912,0.0,0.0,0.0,14.844025,14.552823,12.237766,...,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0,9,4e3c3897-14ce-11ea-bce5-f49634744a41
2,34.26,-15.89,0.0,565.728343,0.0,0.0,0.0,14.844025,14.552823,12.237766,...,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0,9,4e3c3898-14ce-11ea-bce5-f49634744a41
3,34.26,-15.88,0.0,443.392774,0.0,0.0,0.0,14.844025,14.552823,12.237766,...,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0,10,4e3c3899-14ce-11ea-bce5-f49634744a41
4,34.26,-15.87,0.0,437.443428,0.0,0.0,0.0,14.844025,14.552823,12.237766,...,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0,10,4e3c389a-14ce-11ea-bce5-f49634744a41


In [38]:
target='target_2015'
categoric_features=['LC_Type1_mode']

numeric_features=[i for i in train.columns if not i in categoric_features and i !=target and  '2019' not in i]
target,categoric_features,numeric_features

('target_2015',
 ['LC_Type1_mode'],
 ['X',
  'Y',
  'elevation',
  'precip 2014-11-16 - 2014-11-23',
  'precip 2014-11-23 - 2014-11-30',
  'precip 2014-11-30 - 2014-12-07',
  'precip 2014-12-07 - 2014-12-14',
  'precip 2014-12-14 - 2014-12-21',
  'precip 2014-12-21 - 2014-12-28',
  'precip 2014-12-28 - 2015-01-04',
  'precip 2015-01-04 - 2015-01-11',
  'precip 2015-01-11 - 2015-01-18',
  'precip 2015-01-18 - 2015-01-25',
  'precip 2015-01-25 - 2015-02-01',
  'precip 2015-02-01 - 2015-02-08',
  'precip 2015-02-08 - 2015-02-15',
  'precip 2015-02-15 - 2015-02-22',
  'precip 2015-02-22 - 2015-03-01',
  'precip 2015-03-01 - 2015-03-08',
  'precip 2015-03-08 - 2015-03-15',
  'Square_ID'])

In [39]:
train.head()

,X,Y,target_2015,elevation,precip 2014-11-16 - 2014-11-23,precip 2014-11-23 - 2014-11-30,precip 2014-11-30 - 2014-12-07,precip 2014-12-07 - 2014-12-14,precip 2014-12-14 - 2014-12-21,precip 2014-12-21 - 2014-12-28,...,precip 2019-03-24 - 2019-03-31,precip 2019-03-31 - 2019-04-07,precip 2019-04-07 - 2019-04-14,precip 2019-04-14 - 2019-04-21,precip 2019-04-21 - 2019-04-28,precip 2019-04-28 - 2019-05-05,precip 2019-05-05 - 2019-05-12,precip 2019-05-12 - 2019-05-19,LC_Type1_mode,Square_ID
0,34.26,-15.91,0.0,887.764222,0.0,0.0,0.0,14.844025,14.552823,12.237766,...,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0,9,4e3c3896-14ce-11ea-bce5-f49634744a41
1,34.26,-15.90,0.0,743.403912,0.0,0.0,0.0,14.844025,14.552823,12.237766,...,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0,9,4e3c3897-14ce-11ea-bce5-f49634744a41
2,34.26,-15.89,0.0,565.728343,0.0,0.0,0.0,14.844025,14.552823,12.237766,...,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0,9,4e3c3898-14ce-11ea-bce5-f49634744a41
3,34.26,-15.88,0.0,443.392774,0.0,0.0,0.0,14.844025,14.552823,12.237766,...,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0,10,4e3c3899-14ce-11ea-bce5-f49634744a41
4,34.26,-15.87,0.0,437.443428,0.0,0.0,0.0,14.844025,14.552823,12.237766,...,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0,10,4e3c389a-14ce-11ea-bce5-f49634744a41


In [40]:
IMAGE_SHAPE=(train.X.nunique(),train.Y.nunique())
IMAGE_SHAPE

(161, 144)

In [41]:
sorted_X=train.sort_values(['X'])
print(sorted_X.shape)
result=np.zeros(IMAGE_SHAPE)
sorted_X.head()
index_through=0
for index, frame in sorted_X.groupby("X"):
    print(frame.shape)
#     result[index_through][:]=frame['elevation']

(16466, 40)
(9, 40)
(14, 40)
(15, 40)
(16, 40)
(17, 40)
(19, 40)
(20, 40)
(21, 40)
(23, 40)
(24, 40)
(25, 40)
(27, 40)
(31, 40)
(33, 40)
(35, 40)
(44, 40)
(50, 40)
(65, 40)
(78, 40)
(81, 40)
(82, 40)
(86, 40)
(87, 40)
(88, 40)
(88, 40)
(90, 40)
(94, 40)
(95, 40)
(96, 40)
(97, 40)
(98, 40)
(100, 40)
(107, 40)
(112, 40)
(120, 40)
(121, 40)
(122, 40)
(122, 40)
(123, 40)
(124, 40)
(125, 40)
(127, 40)
(129, 40)
(129, 40)
(130, 40)
(130, 40)
(130, 40)
(131, 40)
(132, 40)
(132, 40)
(133, 40)
(135, 40)
(138, 40)
(139, 40)
(139, 40)
(140, 40)
(140, 40)
(141, 40)
(142, 40)
(144, 40)
(144, 40)
(144, 40)
(144, 40)
(144, 40)
(144, 40)
(144, 40)
(144, 40)
(144, 40)
(144, 40)
(144, 40)
(144, 40)
(144, 40)
(144, 40)
(144, 40)
(144, 40)
(144, 40)
(144, 40)
(144, 40)
(144, 40)
(144, 40)
(144, 40)
(144, 40)
(144, 40)
(144, 40)
(144, 40)
(144, 40)
(144, 40)
(144, 40)
(143, 40)
(140, 40)
(137, 40)
(134, 40)
(133, 40)
(131, 40)
(131, 40)
(129, 40)
(128, 40)
(128, 40)
(127, 40)
(126, 40)
(119, 40)
(116, 40)


In [42]:
#[k for k in train.columns if "prec"in k and "2019" not in k]

In [43]:
train[[k for k in train.columns if "prec"in k and "2019" not in k]].values.reshape(1,-1)[0]

array([ 0.        ,  0.        ,  0.        , ...,  9.36976308,
       21.42813075,  2.49368298])

In [44]:
from sklearn.preprocessing import  MinMaxScaler as MinMax
train=pd.read_csv('/kaggle/input/floodmalawi/Train.csv')

scaler_precip=MinMax().fit(X=train[[k for k in train.columns if "prec"in k and "2019" not in k]].values)
train[[k for k in train.columns if "prec"in k and "2019" not in k]]=scaler_precip.transform(train[[k for k in train.columns if "prec"in k and "2019" not in k]])
scaler_elevation=MinMax().fit(X=train['elevation'].values.reshape(-1,1))
train["elevation"]=scaler_elevation.transform(train['elevation'].values.reshape(-1,1))
#using the mean of the food on each type(not realy appropriiate if we suppose that the near condition is the main reason for flooding. )

for k , frame in train.groupby("LC_Type1_mode"):
    print(frame.mean()[target],k)
    train.loc[train.LC_Type1_mode==k,'LC_Type1_mode']=frame.mean()[target]
    
train.head()


0.034039530709836056 2
0.00694239041221374 4
0.017092908178846154 5
0.0 7
0.07811048552857143 8
0.02885364479895209 9
0.07432715167948928 10
0.39936223745602395 11
0.11086638899964228 12
0.07489592539047621 13
0.06028378107822012 14
0.007639637918819189 17


,X,Y,target_2015,elevation,precip 2014-11-16 - 2014-11-23,precip 2014-11-23 - 2014-11-30,precip 2014-11-30 - 2014-12-07,precip 2014-12-07 - 2014-12-14,precip 2014-12-14 - 2014-12-21,precip 2014-12-21 - 2014-12-28,...,precip 2019-03-24 - 2019-03-31,precip 2019-03-31 - 2019-04-07,precip 2019-04-07 - 2019-04-14,precip 2019-04-14 - 2019-04-21,precip 2019-04-21 - 2019-04-28,precip 2019-04-28 - 2019-05-05,precip 2019-05-05 - 2019-05-12,precip 2019-05-12 - 2019-05-19,LC_Type1_mode,Square_ID
0,34.26,-15.91,0.0,0.305401,0.0,0.0,0.0,0.769368,0.563732,0.535691,...,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0,0.028854,4e3c3896-14ce-11ea-bce5-f49634744a41
1,34.26,-15.90,0.0,0.253054,0.0,0.0,0.0,0.769368,0.563732,0.535691,...,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0,0.028854,4e3c3897-14ce-11ea-bce5-f49634744a41
2,34.26,-15.89,0.0,0.188626,0.0,0.0,0.0,0.769368,0.563732,0.535691,...,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0,0.028854,4e3c3898-14ce-11ea-bce5-f49634744a41
3,34.26,-15.88,0.0,0.144266,0.0,0.0,0.0,0.769368,0.563732,0.535691,...,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0,0.074327,4e3c3899-14ce-11ea-bce5-f49634744a41
4,34.26,-15.87,0.0,0.142109,0.0,0.0,0.0,0.769368,0.563732,0.535691,...,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0,0.074327,4e3c389a-14ce-11ea-bce5-f49634744a41


In [45]:
train.LC_Type1_mode.unique()

array([0.02885364, 0.07432715, 0.07811049, 0.00694239, 0.11086639,
       0.01709291, 0.07489593, 0.06028378, 0.03403953, 0.39936224,
       0.00763964, 0.        ])

In [ ]:
#TODO: scalling values(keep in my that value from different columns will be compare one to another)
#TODO:adding  elevation, type of soil. 
#TODO ; data augmentation for flood exemple

x_range=train.X.unique()
y_range=train.Y.unique()
import sys
from joblib import parallel_backend
from joblib import Parallel, delayed


NUM_TIME=len([k for k in train.columns if "prec"in k and "2019" not in k])
TIME_COL=[k for k in train.columns if "prec"in k and "2019" not in k]+["elevation", 'LC_Type1_mode']+[target]
PADDING=3


# index_table=np.full((len(x_range),len(y_range)),-1)
matrix=np.full((NUM_TIME+3,len(x_range),len(y_range)),-1.)

def fill_matrix(indx,x,matrix=matrix):
    x_range=train.X.unique()
    y_range=train.Y.unique()
    counter=0
    for indy,y in enumerate(y_range):
#         print(train.loc[(train.X==round(x,2))& (train.Y==round(y,2)), ['elevation','target_2015']])
        try:
            feat=train.loc[(train.X==round(x,2))& (train.Y==round(y,2)), TIME_COL]
            for ind,col in enumerate(TIME_COL):
#                 print(indx,ind,y,'/')
                print(feat[col].values[0])
                matrix[ind][indx][indy]=feat[col].values[0]
#                 print(feat.values[0])
            

        except Exception as inst:
            print("Unexpected error:", inst)
#             print(index_table)
            pass
    
    return counter   
    
Parallel(require='sharedmem',n_jobs=-1)(delayed(fill_matrix)(indx,x)for indx,x in enumerate(x_range))


0.0
0.0
0.0
0.769368339242944
0.5637321495973231
0.5356912528900217
0.909368160100859
0.40729090435880705
0.0
0.13705649751502075
0.37912393646892484
0.6311781717541791
0.2747509449557288
0.0
0.0
0.0
0.769368339242944
0.5637321495973231
0.044375462190654705
0.3284267277952757
0.0009381538794220046
0.0
0.2671776115613329
0.5356912528900217
0.909368160100859
0.40729090435880705
0.0
0.13705649751502075
0.37912393646892484
0.6311781717541791
0.2747509449557288
0.044375462190654705
0.3284267277952757
0.0009381538794220046
0.0
0.3054007984620804
0.02885364479895209
0.0
0.0
0.02885364479895209
0.0
0.0
0.0
0.0
0.769368339242944
0.0
0.5637321495973231
0.5356912528900217
0.909368160100859
0.40729090435880705
0.0
0.13705649751502075
0.37912393646892484
0.6311781717541791
0.2747509449557288
0.044375462190654705
0.3284267277952757
0.0009381538794220046
0.0
0.264864072947552
0.02885364479895209
0.0
0.0
0.769368339242944
0.5637321495973231
0.5356912528900217
0.909368160100859
0.40729090435880705
0.0


Unexpected error: index 0 is out of bounds for axis 0 with size 0
Unexpected error: index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
Unexpected error: index 0 is out of bounds for axis 0 with size 0
Unexpected error: Unexpected error: index 0 is out of bounds for axis 0 with size 0
Unexpected error: index 0 is out of bounds for axis 0 with size 0
Unexpected error: index 0 is out of bounds for axis 0 with size 0
Unexpected error:index 0 is out of bounds for axis 0 with size 0
 index 0 is out of bounds for axis 0 with size 0
Unexpected error: index 0 is out of bounds for axis 0 with size 0
Unexpected error: index 0 is out of bounds for axis 0 with size 0
Unexpected error: index 0 is out of bounds for axis 0 with size 0
Unexpected error: index 0 is out of bounds for axis 0 with size 0
Unexpected error:Unexpected error: index 0 is out of bounds for axis 0 with size 0 index 0 is out of bounds for axis 0 with size 0
Unexpected error: index 0 is 

index 0 is out of bounds for axis 0 with size 0
Unexpected error: index 0 is out of bounds for axis 0 with size 0
Unexpected error: index 0 is out of bounds for axis 0 with size 0
Unexpected error: index 0 is out of bounds for axis 0 with size 0
Unexpected error: Unexpected error: index 0 is out of bounds for axis 0 with size 0
Unexpected error: index 0 is out of bounds for axis 0 with size 0
Unexpected error: index 0 is out of bounds for axis 0 with size 0
Unexpected error: index 0 is out of bounds for axis 0 with size 0
Unexpected error: index 0 is out of bounds for axis 0 with size 0
Unexpected error: index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
Unexpected error: index 0 is out of bounds for axis 0 with size 0
Unexpected error: index 0 is out of bounds for axis 0 with size 0
Unexpected error: index 0 is out of bounds for axis 0 with size 0
Unexpected error: index 0 is out of bounds for axis 0 with size 0
Unexpected error: index 0 is

0.5356912528900217
0.0
0.0
0.769368339242944
0.5637321495973231
0.5356912528900217
0.909368160100859
0.40729090435880705
0.0
0.13705649751502075
0.37912393646892484
0.6311781717541791
0.2747509449557288
0.044375462190654705
0.3284267277952757
0.0009381538794220046
0.0
0.10936571089803263
0.07432715167948928
0.0

0.909368160100859
0.40729090435880705
0.0
0.13705649751502075
0.37912393646892484
0.6311781717541791
0.2747509449557288
0.044375462190654705
0.3284267277952757
0.0009381538794220046
0.0
0.1162294584470267
0.07432715167948928
0.0
0.0
0.0
0.0
0.769368339242944
0.5637321495973231
0.5356912528900217
0.909368160100859
0.40729090435880705
0.0
0.13705649751502075
0.37912393646892484
0.6311781717541791
0.2747509449557288
0.044375462190654705
0.3284267277952757
0.0009381538794220046
0.0
0.11070215262918354
0.07432715167948928
0.0
0.0
0.0
0.0
0.769368339242944
0.5637321495973231
0.5356912528900217
0.909368160100859
0.40729090435880705
0.0
0.13705649751502075
0.37912393646892484
0.6311781

Unexpected error:index 0 is out of bounds for axis 0 with size 0
Unexpected error:  index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
Unexpected error: index 0 is out of bounds for axis 0 with size 0
Unexpected error: Unexpected error: index 0 is out of bounds for axis 0 with size 0
Unexpected error: index 0 is out of bounds for axis 0 with size 0
Unexpected error: index 0 is out of bounds for axis 0 with size 0index 0 is out of bounds for axis 0 with size 0
Unexpected error: index 0 is out of bounds for axis 0 with size 0
Unexpected error: 
Unexpected error: index 0 is out of bounds for axis 0 with size 0
Unexpected error: index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
Unexpected error: Unexpected error: index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
Unexpected error: index 0 is out of bounds for axis 0 with size 0
Unexpected error: Unexpected

...

0.07388265034462564
0.0
0.0
0.056767621193545684
0.5670258202344364
0.2718458299971822
0.5415175695145272
0.40722065116405387
0.27437553633707346
0.7016822315812259
0.34249952748147106
0.209253116446307
0.9999999999999998
0.17254088490148978
0.3020998938825128
0.15990740845190943
0.0
0.5461680567371685
0.02885364479895209
0.0
0.011414025107849009
0.1559731936908321
0.036813221246261046
0.0
0.11513029682593975
0.07432715167948928
0.0
0.0
0.0
0.0
0.46178814483774866
0.3388483793763809
0.24023860497489954
0.5330692250699635
0.05379931980150038
0.1860589961826017
0.04231901692809882
0.5271019530036377
0.7738006782772622
0.499978099939508
0.048379812639140665
0.2360374366182619
0.04508436085770118
0.0
0.09803370257926633
0.02885364479895209
0.0
0.0
0.0
0.0
0.46178814483774866
0.3388483793763809
0.24023860497489954
0.5330692250699635
0.05379931980150038
0.18605899618260170.049548312637998
0.0
0.0
0.9508958812434241
0.09309717346259769
0.5362430903992752
0.0
0.8844180176813721
0.2074552279293

Unexpected error: index 0 is out of bounds for axis 0 with size 0
Unexpected error: index 0 is out of bounds for axis 0 with size 0

0.05276892436478328
0.07432715167948928
0.0
0.0
0.0
0.0
0.6293644891121267
0.2709217094469616
0.8204204141940488
0.2225207219665549
0.5189783058294661
0.25523327756966674
0.0
0.5629513497521061
0.6571972742858072
0.1416883822186344
0.10310920670871426
0.43351074247463983
0.2348210358614672
0.0
0.05085220005169716
0.07432715167948928
0.0
Unexpected error: index 0 is out of bounds for axis 0 with size 0
0.0
0.0
0.0
0.6293644891121267
0.2709217094469616
0.8204204141940488
0.2225207219665549
0.5189783058294661
0.25523327756966674
0.0
0.5629513497521061
0.6571972742858072
0.1416883822186344
0.10310920670871426
0.43351074247463983
0.2348210358614672
0.0
0.045754119092125276
0.07432715167948928
0.0
Unexpected error: index 0 is out of bounds for axis 0 with size 0
Unexpected error: index 0 is out of bounds for axis 0 with size 0
0.0
0.0
0.0
0.6293644891121267
0.2

0.2348210358614672
0.0
0.00.05457820190066538
0.07432715167948928
0.0
0.0
0.0
0.0

0.11487616635242459
0.02885364479895209
0.0
0.0
0.0
0.0
0.46178814483774866
0.3388483793763809
0.6293644891121267
0.2709217094469616
0.8204204141940488
0.240238604974899540.2225207219665549
0.5189783058294661
0.25523327756966674
0.0
0.5629513497521061
0.6571972742858072
0.1416883822186344
0.10310920670871426
0.43351074247463983
0.2348210358614672
0.0
0.03614740592435299
0.07432715167948928
0.1557208419

0.5330692250699635
0.05379931980150038
0.1860589961826017
0.04231901692809882
0.5271019530036377
0.7738006782772622
0.499978099939508
0.048379812639140665
0.2360374366182619
0.04508436085770118
0.0
0.11704607255317645
0.02885364479895209
0.0
0.0
0.0
0.0
0.46178814483774866
0.0
0.0
0.0
0.33884837937638090.6293644891121267
0.2709217094469616
0.8204204141940488
0.2225207219665549
0.5189783058294661
0.25523327756966674
0.0
0.5629513497521061
0.6571972742858072
0.1416883822186344
0.10310920670871426
0.43351074

0.00.036813221246261046
0.0
0.0
0.46178814483774866
0.3388483793763809
0.24023860497489954
0.5330692250699635
0.05379931980150038
0.1860589961826017
0.04231901692809882
0.5271019530036377
0.7738006782772622
0.499978099939508
0.048379812639140665
0.2360374366182619
0.04508436085770118
0.0
0.09053466333183627
0.02885364479895209
0.0

0.0
0.1496945143143911
0.07432715167948928
0.0
0.0
0.0
0.0
0.0
0.0
0.562546073357552
0.2980063942909683
0.5890820978585346
0.34634702383745397
0.33349554557941125
0.22765112739633486
0.016874038396806183
0.0
0.5486569700367098
0.7036910366349771
0.2845507380165899
0.08128672679797211
0.3547713863417403
0.15916653886719773
0.0
0.08857115481184863
0.02885364479895209
0.0
0.0
0.238863405794017
0.37917835223690266
0.0
0.7978216630275594
0.2936046973961563
0.13856064532677081
0.2859394541611663
0.0
0.0
0.46178814483774866
0.5680140317892846
0.974858407627883
0.6908966672110797
0.011414025107849009
0.1559731936908321
0.036813221246261046
0.0
0.15989638227261757
0.

0.0
0.0
0.0
0.00.6293644891121267
0.2709217094469616
0.8204204141940488
0.2225207219665549
0.5189783058294661
0.25523327756966674
0.0
0.5629513497521061

0.0
0.21816143088776024
0.6282614122531966
0.3673810783388646
0.6282193570095858
0.21707369648290914
0.20578041985560258
0.6571972742858072
0.1416883822186344
0.10310920670871426
0.43351074247463983
0.2348210358614672
0.0
0.03599283138557127
0.07432715167948928
0.0
0.4973907886274971
0.4636856009210365
0.3852888694013295
0.7366074574170647
0.16388346632579165
0.454287134304871
0.08143558294945241
0.0
0.24041601966712392
0.02885364479895209
0.0
0.15196741170497957
0.0
0.0
0.0
0.0
0.21816143088776024
0.6282614122531966
0.3673810783388646
0.6282193570095858
0.21707369648290914
0.20578041985560258
0.4973907886274971
0.4636856009210365
0.3852888694013295
0.7366074574170647
0.16388346632579165
0.454287134304871
0.08143558294945241
0.0
0.24614401284890494
0.02885364479895209
0.0
0.15196741170497957
0.0
0.238863405794017
0.37917835223690266
0

In [ ]:
np.pad(matrix,((0,0),(5,5),(5,5))).shape,matrix.shape

In [ ]:

def create_data(PADDING=5):
    NUM_TIME=len([k for k in train.columns if "prec"in k and "2019" not in k])
    data=np.zeros((train.shape[0],NUM_TIME,3,PADDING*2+1,PADDING*2+1))
    matrixpad=np.pad(matrix,((0,0),(PADDING,PADDING),(PADDING,PADDING)))
    labels=np.array([])
    for tim in range(NUM_TIME):
        index_sample=0
        for indx,x in enumerate(x_range):
            for indy,y in enumerate(y_range):
                if(matrix[-1][indx][indy]!=-1):
                    print(tim,indy,indx,index_sample)
                    data[index_sample][tim][0][:][:]=matrixpad[tim][indx:indx+2*PADDING+1,indy:indy+2*PADDING+1]
                    data[index_sample][tim][1][:][:]=matrixpad[-2][indx:indx+2*PADDING+1,indy:indy+2*PADDING+1]
                    data[index_sample][tim][2][:][:]=matrixpad[-3][indx:indx+2*PADDING+1,indy:indy+2*PADDING+1]
                    
#                     print(matrixpad[-2][indx:indx+2*PADDING+1,indy:indy+2*PADDING+1])
                    index_sample+=1
    for indx,x in enumerate(x_range):
            for indy,y in enumerate(y_range):
                if(matrix[-1][indx][indy]!=-1):
                    labels=np.append(labels,matrix[-1][indx][indy])
    
    return data,labels
data,labels=create_data(PADDING)

In [ ]:
plt.figure(figsize=(8,9))
plt.imshow(data[500][5][0])

In [ ]:
matrixpad=np.pad(matrix[-3],PADDING)
plt.imshow(matrixpad)


In [ ]:
matrixpad[-2]

Every row doesnt have data. 
<br> We going to create a shape around each square to take a crop around that square. 



this is the general shape of the area.

In [ ]:
figure=plt.figure(figsize=(8,10))
plt.scatter(train["X"].values,train["Y"].values,train['elevation'])

In [ ]:
import itertools
import sys


So we can construct a 3d image (precip,elevation,sol type)

If data_format='channels_first' 5D tensor with shape: (samples, time, channels, rows, cols)

In [ ]:
# PADDING=1
NUM_TIME=len([k for k in train.columns if "prec"in k and "2019" not in k])

In [ ]:
set(labels)

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedShuffleSplit

rs = ShuffleSplit(n_splits=3, test_size=.25, random_state=0)
for train_index, test_index in rs.split(data):
    print(train_index,test_index)

    
Dataset_tf_train=tf.data.Dataset.from_tensor_slices((data[train_index],labels[train_index]))
Dataset_tf_test=tf.data.Dataset.from_tensor_slices((data[test_index],labels[test_index]))

    


In [ ]:
shuffle_buffer=100
batch_size=32
Dataset_tf_train = Dataset_tf_train.shuffle(shuffle_buffer)
Dataset_tf_train = Dataset_tf_train.batch(batch_size).prefetch(1)
Dataset_tf_test = Dataset_tf_test.shuffle(shuffle_buffer)
Dataset_tf_test = Dataset_tf_test.batch(batch_size).prefetch(1)

In [ ]:
len(labels[labels==1])

In [ ]:
import tensorflow.keras as keras
import os

pos=len(labels[labels==1])
neg=len(labels[labels==0])
total=pos+neg
initial_bias = np.log([pos/neg])

def define_model(output_bias=None,learning_rate=0.01):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)

    model = keras.models.Sequential([

    keras.layers.ConvLSTM2D(15, (3,3),data_format='channels_first',return_sequences=True, stateful=False,
                             ),
    keras.layers.MaxPooling3D(pool_size=(2, 2, 2), padding='same', data_format='channels_first'),
    keras.layers.ConvLSTM2D(20, (3,3),data_format='channels_first',return_sequences=True, stateful=False,
                             ),
    keras.layers.MaxPooling3D(pool_size=(2, 3, 3), padding='same', data_format='channels_first'),
    keras.layers.Flatten(),
    keras.layers.Dense(80),
    
    keras.layers.Dense(1,bias_initializer=output_bias),
#     keras.layers.Lambda(lambda x: x * 2.0)
    ])
    optimizer = keras.optimizers.Nadam(learning_rate=0.01)
    model.compile(loss=keras.losses.Huber(),
                  optimizer=optimizer,
                  metrics=["mae",tf.keras.metrics.RootMeanSquaredError()])
    return model

model=define_model()
model.predict(np.full((1,NUM_TIME,3,PADDING*2+1,PADDING*2+1),1.))




In [ ]:
model.summary()

In [ ]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()
        

In [ ]:
model_checkpoint = keras.callbacks.ModelCheckpoint(
    "my_checkpoint.h5", save_best_only=True)
early_stopping = keras.callbacks.EarlyStopping(patience=5)

In [ ]:
lr_schedule = keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 2))

history=model.fit(Dataset_tf_train,epochs=20,callbacks=[early_stopping,lr_schedule],
          validation_data=Dataset_tf_test)


In [ ]:

plt.semilogx(history.history["lr"], history.history["loss"])




In [ ]:
tf.random.set_seed(42)
np.random.seed(42)
#for repetability

lr_schedule = keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-1 * 10**(-epoch / 5))


model.fit(Dataset_tf_train,epochs=70,callbacks=[early_stopping, model_checkpoint],
          validation_data=Dataset_tf_test,
         
         )

In [ ]:
del matrix
del data

# Submit

In [ ]:
NUM_TIME,NUM_TIME_sub

In [ ]:
train.X.unique(),train.Y.unique()

In [ ]:
NUM_TIME_sub=len([k for k in train.columns if "prec"in k and "2014" not in k and '2015' not in k])
TIME_COL_sub=[k for k in train.columns if "prec"in k and "2014" not in k and '2015' not in k]+['elevation', 'LC_Type1_mode']+["Square_ID"]

# index_table=np.full((len(x_range),len(y_range)),-1)
matrix_sub=np.full((NUM_TIME_sub+3,len(x_range),len(y_range)),-1.,dtype=object)

def fill_matrix_sub(indx,x,matrix=matrix_sub):
    x_range=train.X.unique()
    y_range=train.Y.unique()
    counter=0
    for indy,y in enumerate(y_range):
        try:
            feat=train.loc[(train.X==round(x,2))& (train.Y==round(y,2)), TIME_COL_sub]
            
            for ind,col in enumerate(TIME_COL_sub):
#                 print(indx,ind,y,'/')
                print(feat[col].values[0])
                matrix_sub[ind][indx][indy]=feat[col].values[0]

        except Exception as inst:
            print('error')
            print("Unexpected error:", inst)
            

    return counter   
    
Parallel(require='sharedmem',n_jobs=-1)(delayed(fill_matrix_sub)(indx,x)for indx,x in enumerate(x_range))




In [ ]:
def create_data_sub(PADDING=PADDING,NUM_TIME=NUM_TIME_sub):
    
    data=np.zeros((train.shape[0],NUM_TIME,2,PADDING*2+1,PADDING*2+1))
    matrixpad=np.pad(matrix,((0,0),(PADDING,PADDING),(PADDING,PADDING)))
    labels=np.array([],dtype=object)
    for tim in range(NUM_TIME):
        index_sample=0
        for indx,x in enumerate(x_range):
            for indy,y in enumerate(y_range):
                if(matrix_sub[-1][indx][indy]!=-1):
                    print(tim,indy,indx,index_sample)
                    data[index_sample][tim][0][:][:]=matrixpad[tim][indx:indx+2*PADDING+1,indy:indy+2*PADDING+1]
                    data[index_sample][tim][1][:][:]=matrixpad[-2][indx:indx+2*PADDING+1,indy:indy+2*PADDING+1]
                    data[index_sample][tim][2][:][:]=matrixpad[-3][indx:indx+2*PADDING+1,indy:indy+2*PADDING+1]
                    index_sample+=1
    for indx,x in enumerate(x_range):
            for indy,y in enumerate(y_range):
                if(matrix_sub[-1][indx][indy]!=-1):
                    print('labels')
                    labels=np.append(labels,matrix_sub[-1][indx][indy])
    

    return data,labels
data_sub,labels_sub=create_data_sub()

In [ ]:
d=model.predict(data_sub)

In [ ]:
sample=pd.read_csv('/kaggle/input/floodmalawi/SampleSubmission.csv')

In [ ]:
labels_sub.reshape((1,-1))[0]


In [ ]:
#todo scaling problem different dimension
#tabulatoin on soil type

to_submit=pd.DataFrame(data={'Square_ID':labels_sub,'target_2019':d.reshape(1,-1)[0]})
to_submit.to_csv('submit.csv',index=False)
to_submit.head()

In [ ]:
data_sub.shape,labels_sub.shape,train.shape